In [1]:
from google.colab import files
records = files.upload()

Saving Data_for_UCI_named.csv to Data_for_UCI_named.csv


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('Data_for_UCI_named.csv')

In [5]:
df.drop('stab', axis = 1, inplace = True)

In [6]:
X = df.drop('stabf', axis = 1 )
y = df[ 'stabf' ] 

In [7]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalised_train_df = scaler.fit_transform(x_train)
normalised_train_df = pd.DataFrame(normalised_train_df,
                                   columns=x_train.columns)
x_test = x_test.reset_index(drop= True )
normalised_test_df = scaler.transform(x_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.columns)

In [18]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
classifier = ExtraTreesClassifier(random_state=1)
classifier.fit(normalised_train_df, y_train)
extra_pred = classifier.predict(normalised_test_df)

accuracy = accuracy_score(y_true=y_test, y_pred=extra_pred)
print( 'Accuracy: {}' .format(round(accuracy,4) ))

Accuracy: 0.928


In [48]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [100, 300, 500, 1000] 
max_features = ['auto', 'log2', None] 
min_samples_split = [2,5,7] 
min_samples_leaf = [4,6,8] 

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}
rf_random = RandomizedSearchCV(estimator = classifier,
                               param_distributions = random_grid,
                               n_iter = 10, cv = 5, verbose=1, random_state=1, n_jobs = -1, scoring = 'accuracy')

In [49]:
rf_random.fit(normalised_train_df, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=XGBClassifier(random_state=1, seed=1),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2', None],
                                        'min_samples_leaf': [4, 6, 8],
                                        'min_samples_split': [2, 5, 7],
                                        'n_estimators': [100, 300, 500, 1000]},
                   random_state=1, scoring='accuracy', verbose=1)

In [50]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 6,
 'max_features': 'log2'}

In [51]:
classifier_opt = ExtraTreesClassifier(random_state=1, n_estimators= 1000, min_samples_split=5, min_samples_leaf = 6, max_features = 'log2')
classifier_opt.fit(normalised_train_df, y_train)
extra_pred_opt = classifier_opt.predict(normalised_test_df)
accuracy_opt = accuracy_score(y_true=y_test, y_pred=extra_pred_opt)
print( 'Accuracy: {}' .format(round(accuracy_opt,4) ))
#Accuracy reduced

Accuracy: 0.9185


In [39]:
we = pd.DataFrame({'weights':classifier_opt.feature_importances_, 'features':x_train.columns})
print('Most important feature is {}'.format(we[we['weights']== we['weights'].max()]['features'].iloc[0]))
print('Least important feature is {}'.format(we[we['weights']== we['weights'].min()]['features'].iloc[0]))

Most important feature is tau1
Least important feature is p1


In [47]:
from xgboost import XGBClassifier
classifier = XGBClassifier(random_state=1,seed=1)
classifier.fit(normalised_train_df, y_train)
boost_pred = classifier.predict(normalised_test_df)

accuracy = accuracy_score(y_true=y_test, y_pred=boost_pred)
print( 'Accuracy: {}' .format(round(accuracy, 4) ))

Accuracy: 0.9195


In [13]:
from lightgbm import LGBMClassifier
classifier = LGBMClassifier(random_state=1)
classifier.fit(normalised_train_df, y_train)
random_pred = classifier.predict(normalised_test_df)

accuracy = accuracy_score(y_true=y_test, y_pred=random_pred)
print( 'Accuracy: {}' .format(round(accuracy,4)))

Accuracy: 0.9375
